In [22]:
import pickle
import numpy as np
import pandas as pd
from utils import prepare_df

# Load ground-truth df and softmax list
result = prepare_df('50salads')
if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# Derive case order from recovery_results_50salads_complete_15.csv
order_df = pd.read_csv('recovery_results_50salads_complete_15.csv', usecols=['case:concept:name'])
case_order = order_df['case:concept:name'].astype(str).drop_duplicates().tolist()

# Helper: collapse consecutive duplicates in a sequence

def collapse_runs(labels):
    collapsed = []
    previous = None
    for label in labels:
        s = str(label)
        if s != previous:
            collapsed.append(s)
            previous = s
    return collapsed

# Load KARI results
with open('kari_results_50salads_complete.pkl', 'rb') as f:
    kari_results = pickle.load(f)

assert len(kari_results) == len(case_order), f"Expected {len(case_order)} results, got {len(kari_results)}"

correct_cases = []
incorrect = []  # tuples of (case_id, pred_tokens, gt_collapsed)

for i, case_id in enumerate(case_order):
    rec = kari_results[i]

    # Predicted sequence may be a space-separated string or list-like
    pred_seq = rec.get('sequence')
    if isinstance(pred_seq, (list, tuple, np.ndarray)):
        pred_tokens = [str(x) for x in pred_seq]
    else:
        pred_tokens = [tok for tok in str(pred_seq).split() if tok != '']

    # Ground truth labels (with runs) → collapse to unique steps
    gt_labels = df.loc[df['case:concept:name'] == case_id, 'concept:name'].tolist()
    gt_collapsed = collapse_runs(gt_labels)

    if pred_tokens == gt_collapsed:
        correct_cases.append(case_id)
    else:
        incorrect.append((case_id, pred_tokens, gt_collapsed))

num_correct = len(correct_cases)
num_total = len(case_order)
accuracy = num_correct / num_total
print(f"KARI sequence accuracy: {num_correct}/{num_total} = {accuracy:.3f}")
if incorrect:
    print("Incorrect cases:", [cid for cid, _, _ in incorrect])


KARI sequence accuracy: 3/40 = 0.075
Incorrect cases: ['30', '17', '9', '8', '20', '7', '23', '5', '28', '1', '13', '36', '33', '14', '10', '31', '22', '34', '38', '37', '6', '24', '27', '21', '15', '11', '19', '16', '12', '32', '25', '35', '39', '26', '29', '4', '18']


In [23]:
# Pretty print predicted vs ground-truth collapsed sequences per case

try:
    kari_results
    df
    case_order
except NameError:
    import pickle
    import numpy as np
    import pandas as pd
    from utils import prepare_df
    result = prepare_df('50salads')
    if len(result) == 2:
        df, softmax_lst = result
    else:
        df, softmax_lst, _ = result
    with open('kari_results_50salads_complete.pkl', 'rb') as f:
        kari_results = pickle.load(f)
    order_df = pd.read_csv('recovery_results_50salads_complete_15.csv', usecols=['case:concept:name'])
    case_order = order_df['case:concept:name'].astype(str).drop_duplicates().tolist()

if 'collapse_runs' not in globals():
    def collapse_runs(labels):
        collapsed = []
        previous = None
        for label in labels:
            s = str(label)
            if s != previous:
                collapsed.append(s)
                previous = s
        return collapsed

rows = []
correct = 0
for i, case_id in enumerate(case_order):
    rec = kari_results[i]
    pred_seq = rec.get('sequence')
    if isinstance(pred_seq, (list, tuple, np.ndarray)):
        pred_tokens = [str(x) for x in pred_seq]
    else:
        pred_tokens = [tok for tok in str(pred_seq).split() if tok != '']
    gt_tokens = collapse_runs(df.loc[df['case:concept:name'] == case_id, 'concept:name'].tolist())
    is_match = (pred_tokens == gt_tokens)
    correct += int(is_match)
    rows.append({
        'case': case_id,
        'status': 'CORRECT' if is_match else 'WRONG',
        'pred': ' '.join(pred_tokens),
        'true': ' '.join(gt_tokens)
    })

print(f"KARI sequence accuracy: {correct}/{len(case_order)} = {correct/len(case_order):.3f}\n")
for r in rows:
    print(f"Case {r['case']}: {r['status']}")
    print(f"  Pred: {r['pred']}")
    print(f"  True: {r['true']}\n")


KARI sequence accuracy: 3/40 = 0.075

Case 30: WRONG
  Pred: 17 11 13 4 5 2 3 0 1 15 8 7 9 6 10 16 18
  True: 17 0 1 11 12 13 2 3 4 5 15 7 8 6 9 10 16 18

Case 17: WRONG
  Pred: 17 6 13 9 7 8 10 2 0 11 12 13 1 13 4 5 14 15 16 18
  True: 17 6 9 7 8 10 2 0 11 12 13 1 13 4 5 14 15 16 18

Case 9: WRONG
  Pred: 17 6 7 8 9 6 10 11 12 13 2 3 0 1 4 5 4 5 15 14 16 14 18
  True: 17 7 8 9 6 10 11 12 11 12 13 2 3 0 1 4 5 4 5 15 16 14 18

Case 8: WRONG
  Pred: 17 6 7 8 9 6 10 11 12 13 2 3 0 1 4 5 4 5 15 16 14 18
  True: 17 7 8 9 6 10 11 12 11 12 13 2 3 0 1 4 5 4 5 15 16 14 18

Case 20: WRONG
  Pred: 17 11 13 8 7 9 6 10 11 13 0 1 0 1 2 3 4 5 15 16 14 18
  True: 17 8 7 9 6 10 11 13 12 13 0 1 0 1 2 3 4 5 15 16 14 18

Case 7: WRONG
  Pred: 17 2 3 0 1 0 1 4 5 4 5 11 12 7 8 6 9 15 14 16 18
  True: 17 2 3 0 1 0 1 15 1 4 5 4 5 11 12 13 12 13 7 8 6 9 15 16 18

Case 23: WRONG
  Pred: 17 8 7 9 6 10 11 13 0 1 0 1 2 3 4 5 15 16 14 18
  True: 17 8 7 9 6 10 11 13 12 13 0 1 0 1 2 3 4 5 15 16 14 18

Case 5: WRONG
 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import pandas as pd
from utils import prepare_df, group_cases_by_trace, compute_accuracies_by_case
from incremental_softmax_recovery import incremental_softmax_recovery

In [4]:
# Configure logging with selective DEBUG for our modules only
logging.basicConfig(
    level=logging.INFO,           # Set root to INFO (reduces third-party noise)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True                    # Force override of any existing handlers (useful in Jupyter)
)

# Enable DEBUG for our specific modules only
our_modules = [
    'classes', 
    'incremental_softmax_recovery', 
    'utils', 
    'conformance_checking',
    'data_processing',
    'petri_model',
    'calibration'
]

for module_name in our_modules:
    logging.getLogger(module_name).setLevel(logging.DEBUG)

# Silence noisy third-party libraries
logging.getLogger('graphviz').setLevel(logging.WARNING)  # Only show warnings/errors from graphviz
logging.getLogger('matplotlib').setLevel(logging.WARNING)  # Silence matplotlib if present
logging.getLogger('PIL').setLevel(logging.WARNING)  # Silence PIL if present

print("✅ Logging configured: DEBUG for our modules, INFO+ for third-party libraries")


✅ Logging configured: DEBUG for our modules, INFO+ for third-party libraries


In [ ]:
# load your DataFrame and softmax list

if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# Define the two groups
group1_cases = ['20', '11', '5', '36', '14', '4', '30', '15', '3', '18']
group2_cases = [str(i) for i in range(len(softmax_lst)) if str(i) not in group1_cases]

# Compute statistics for both groups
from utils import compute_group_statistics
group_stats = compute_group_statistics(df, softmax_lst, group1_cases, group2_cases)

print("Group 1 Statistics:")
print(f"Number of cases: {group_stats['group1']['num_cases']}")
print(f"Mean argmax accuracy: {group_stats['group1']['mean_accuracy']:.4f}")
print(f"Individual accuracies: {[f'{acc:.4f}' for acc in group_stats['group1']['individual_accuracies']]}")
print()

print("Group 2 Statistics:")
print(f"Number of cases: {group_stats['group2']['num_cases']}")
print(f"Mean argmax accuracy: {group_stats['group2']['mean_accuracy']:.4f}")
print(f"Individual accuracies: {[f'{acc:.4f}' for acc in group_stats['group2']['individual_accuracies']]}")
print()

print(f"Difference (Group 1 - Group 2): {group_stats['group1']['mean_accuracy'] - group_stats['group2']['mean_accuracy']:.4f}")

# group by trace and inspect
trace_groups = group_cases_by_trace(df)
trace_groups

Group 1 Statistics:
Number of cases: 10
Mean argmax accuracy: 0.8295
Individual accuracies: ['0.9056', '0.8281', '0.7344', '0.6779', '0.9259', '0.7252', '0.9428', '0.9250', '0.9448', '0.6858']

Group 2 Statistics:
Number of cases: 30
Mean argmax accuracy: 0.8247
Individual accuracies: ['0.9131', '0.9325', '0.9416', '0.7408', '0.7394', '0.8180', '0.8247', '0.8243', '0.9086', '0.9235', '0.6568', '0.6757', '0.6901', '0.9114', '0.9397', '0.9401', '0.8558', '0.8650', '0.8573', '0.8553', '0.9295', '0.9371', '0.9439', '0.7598', '0.7548', '0.7596', '0.7575', '0.6808', '0.6981', '0.7063']

Difference (Group 1 - Group 2): 0.0048


,case_list,trace_length
0,"[0, 1, 2, 3]",5687
1,"[32, 33, 34, 35]",6186
2,"[36, 37, 38, 39]",5840
3,"[28, 29, 30, 31]",5261
4,"[4, 5, 6, 7]",6208
5,"[16, 17, 18, 19]",6293
6,"[24, 25, 26, 27]",6046
7,"[8, 9, 10, 11]",6584
8,"[12, 13, 14, 15]",5558
9,"[20, 21, 22, 23]",5792


In [ ]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [ ]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [ ]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [ ]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [ ]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [ ]:
# Configuration for incremental_softmax_recovery (conformance-only)
config = {
    # === Data Splitting ===
    'n_train_traces': 10,
    'n_test_traces': 10,
    'train_cases': None,
    'test_cases': None,
    'ensure_train_variant_diversity': True,
    'ensure_test_variant_diversity': False,

    # === Sampling Configuration ===
    'sequential_sampling': True,   # True: sample from activity runs, False: uniform sampling
    'n_indices': None,             # Required when sequential_sampling=False
    'n_per_run': 10000,                # Required when sequential_sampling=True
    'independent_sampling': True,

    # === Conformance Parameters ===
    'prob_threshold': 1e-6,                    # Probability threshold for filtering activities
    'chunk_size': 11,                          # Window size for chunked conformance
    'conformance_switch_penalty_weight': 1.0,  # Multiplier for label-switch penalty (actual penalty = weight * P(stay_with_prev_activity))
    'max_hist_len': 3,                         # History length for probability dict
    'merge_mismatched_boundaries': False,      # Merge adjacent chunks when boundary labels disagree
    

    # === Cost Function ===
    'cost_function': "linear",    # "linear", "logarithmic", or callable
    'model_move_cost': 1.0,
    'log_move_cost': 1.0,
    'tau_move_cost': 0.0,
    'non_sync_penalty': 1.0,

    # === Temperature Calibration ===
    'use_calibration': True,
    'temp_bounds': (1.0, 10.0),
    'temperature': None,

    # === Logging ===
    'verbose': True,
    'log_level': logging.INFO,

    # === Miscellaneous ===
    'round_precision': 2,
    'random_seed': 101,
    'save_model_path': "./discovered_petri_net",
    'save_model': True,
}

# Usage:
output = incremental_softmax_recovery(
    df=df,
    softmax_lst=softmax_lst,
    **config 
)

# Unpack results
results_df, accuracy_dict, prob_dict = output

2025-09-17 16:23:15,081 - incremental_softmax_recovery - INFO - Starting incremental softmax recovery (conformance-only).
2025-09-17 16:23:15,089 - incremental_softmax_recovery - INFO - Validated sequential case IDs (found 40 unique cases) and 40 softmax matrices.
2025-09-17 16:23:15,089 - incremental_softmax_recovery - INFO - Validated sampling parameters: sequential runs with n_per_run=10000.
2025-09-17 16:23:15,091 - incremental_softmax_recovery - INFO - Validated input parameters: round_precision=2, prob_threshold=1e-06.
2025-09-17 16:23:15,091 - incremental_softmax_recovery - INFO - Prepared cost function: linear (model=1.0, log=1.0, tau=0.0).
2025-09-17 16:23:15,092 - incremental_softmax_recovery - INFO - Prepared softmax arrays: 40 traces with individual shape (19, 5687).


2025-09-17 16:23:15,712 - incremental_softmax_recovery - INFO - Filtered log and softmax matrices: 237820 -> 237820 events (100.0% retained).
2025-09-17 16:23:16,277 - incremental_softmax_recovery - INFO - Performed train/test split: 10 train cases, 10 test cases.


['20' '11' '5' '36' '14' '4' '30' '15' '3' '18']
Train Case 22: 5792 events
Train Case 29: 5261 events
Train Case 37: 5840 events
Train Case 2: 5687 events
Train Case 12: 5558 events
Train Case 35: 6186 events
Train Case 19: 6293 events
Train Case 26: 6046 events
Train Case 10: 6584 events
Train Case 6: 6208 events
--------------------------------
Test Case 20: 5792 events
Test Case 11: 6584 events
Test Case 5: 6208 events
Test Case 36: 5840 events
Test Case 14: 5558 events
Test Case 4: 6208 events
Test Case 30: 5261 events
Test Case 15: 5558 events
Test Case 3: 5687 events
Test Case 18: 6293 events


In [ ]:
import numpy as np
# ['20' '11' '5' '36' '14' '4' '30' '15' '3' '18']

output = [elem.T if hasattr(elem, 'shape') and isinstance(elem, np.ndarray) else elem for elem in output]


(5792, 19) <class 'numpy.ndarray'>
(6584, 19) <class 'numpy.ndarray'>
(6208, 19) <class 'numpy.ndarray'>
(5840, 19) <class 'numpy.ndarray'>
(5558, 19) <class 'numpy.ndarray'>
(6208, 19) <class 'numpy.ndarray'>
(5261, 19) <class 'numpy.ndarray'>
(5558, 19) <class 'numpy.ndarray'>
(5687, 19) <class 'numpy.ndarray'>
(6293, 19) <class 'numpy.ndarray'>


In [17]:
# Analysis of Results from CSV File
# This cell analyzes the recovery_results_50salads_complete_15.csv file
from utils import compute_group_accuracies_from_results
from typing import List

def analyze_results_from_csv(csv_path: str, group1_cases: List[str], group2_cases: List[str]):
    """Analyze SKTR and argmax performance for two groups from CSV results"""

    print("=" * 80)
    print("ANALYSIS OF RESULTS FROM CSV FILE")
    print("=" * 80)
    print(f"Loading results from: {csv_path}")

    # Load CSV file
    results_df = pd.read_csv(csv_path)
    print(f"Loaded {len(results_df)} rows from CSV file")
    print(f"Unique cases in results: {results_df['case:concept:name'].nunique()}")

    # Define groups
    all_cases = [str(i) for i in range(results_df['case:concept:name'].nunique())]

    # Compute group accuracies
    group_stats = compute_group_accuracies_from_results(results_df, group1_cases, group2_cases)

    # Print results
    print("\n" + "=" * 60)
    print("GROUP COMPARISON RESULTS")
    print("=" * 60)

    group1 = group_stats['group1']
    group2 = group_stats['group2']

    print("\n1. SKTR ALGORITHM PERFORMANCE:")
    print("-" * 40)
    print(f"Group 1 - Mean SKTR accuracy: {group1['sktr_accuracy']:.4f}")
    print(f"Group 2 - Mean SKTR accuracy: {group2['sktr_accuracy']:.4f}")
    print(f"SKTR difference: {group1['sktr_accuracy'] - group2['sktr_accuracy']:.4f}")

    print("\n2. ARGMAX ALGORITHM PERFORMANCE:")
    print("-" * 40)
    print(f"Group 1 - Mean argmax accuracy: {group1['argmax_accuracy']:.4f}")
    print(f"Group 2 - Mean argmax accuracy: {group2['argmax_accuracy']:.4f}")
    print(f"Argmax difference: {group1['argmax_accuracy'] - group2['argmax_accuracy']:.4f}")

    print("\n3. ALGORITHM COMPARISON WITHIN GROUPS:")
    print("-" * 40)
    print(f"Group 1 - SKTR vs Argmax: {group1['sktr_accuracy'] - group1['argmax_accuracy']:.4f}")
    print(f"Group 2 - SKTR vs Argmax: {group2['sktr_accuracy'] - group2['argmax_accuracy']:.4f}")

    print("\n4. TOP 5 INDIVIDUAL PERFORMANCES:")
    print("-" * 40)

    if group1['sktr_individual']:
        print("Group 1 SKTR (highest to lowest):")
        for i, acc in enumerate(sorted(group1['sktr_individual'], reverse=True)[:5]):
            print(f"  {i+1}. {acc:.4f}")

    if group1['argmax_individual']:
        print("Group 1 Argmax (highest to lowest):")
        for i, acc in enumerate(sorted(group1['argmax_individual'], reverse=True)[:5]):
            print(f"  {i+1}. {acc:.4f}")

    if group2['sktr_individual']:
        print("Group 2 SKTR (highest to lowest):")
        for i, acc in enumerate(sorted(group2['sktr_individual'], reverse=True)[:5]):
            print(f"  {i+1}. {acc:.4f}")

    if group2['argmax_individual']:
        print("Group 2 Argmax (highest to lowest):")
        for i, acc in enumerate(sorted(group2['argmax_individual'], reverse=True)[:5]):
            print(f"  {i+1}. {acc:.4f}")

    print("\n5. SUMMARY STATISTICS:")
    print("-" * 40)
    print(f"Group 1 cases: {group1['num_cases']}")
    print(f"Group 2 cases: {group2['num_cases']}")

    # Additional summary: case-level mean ± std and micro accuracy
    import numpy as np
    print("\n6. CASE-LEVEL MEAN ± STD:")
    print("-" * 40)
    if group1['sktr_individual']:
        g1_sktr_mean = np.mean(group1['sktr_individual']); g1_sktr_std = np.std(group1['sktr_individual'])
    else:
        g1_sktr_mean = float('nan'); g1_sktr_std = float('nan')
    if group1['argmax_individual']:
        g1_arg_mean = np.mean(group1['argmax_individual']); g1_arg_std = np.std(group1['argmax_individual'])
    else:
        g1_arg_mean = float('nan'); g1_arg_std = float('nan')
    if group2['sktr_individual']:
        g2_sktr_mean = np.mean(group2['sktr_individual']); g2_sktr_std = np.std(group2['sktr_individual'])
    else:
        g2_sktr_mean = float('nan'); g2_sktr_std = float('nan')
    if group2['argmax_individual']:
        g2_arg_mean = np.mean(group2['argmax_individual']); g2_arg_std = np.std(group2['argmax_individual'])
    else:
        g2_arg_mean = float('nan'); g2_arg_std = float('nan')

    print(f"Group 1 SKTR:   {g1_sktr_mean:.4f} ± {g1_sktr_std:.4f}")
    print(f"Group 1 Argmax: {g1_arg_mean:.4f} ± {g1_arg_std:.4f}")
    print(f"Group 2 SKTR:   {g2_sktr_mean:.4f} ± {g2_sktr_std:.4f}")
    print(f"Group 2 Argmax: {g2_arg_mean:.4f} ± {g2_arg_std:.4f}")

    print("\n7. MICRO ACCURACY (OVER FRAMES):")
    print("-" * 40)
    g1_sub = results_df[results_df['case:concept:name'].isin(group1_cases)]
    g2_sub = results_df[results_df['case:concept:name'].isin(group2_cases)]
    g1_sktr_micro = (g1_sub['sktr_activity'] == g1_sub['ground_truth']).mean()
    g1_arg_micro  = (g1_sub['argmax_activity'] == g1_sub['ground_truth']).mean()
    g2_sktr_micro = (g2_sub['sktr_activity'] == g2_sub['ground_truth']).mean()
    g2_arg_micro  = (g2_sub['argmax_activity'] == g2_sub['ground_truth']).mean()
    print(f"Group 1 SKTR micro:   {g1_sktr_micro:.4f}")
    print(f"Group 1 Argmax micro: {g1_arg_micro:.4f}")
    print(f"Group 2 SKTR micro:   {g2_sktr_micro:.4f}")
    print(f"Group 2 Argmax micro: {g2_arg_micro:.4f}")

    # Overall insights
    print("\n6. KEY INSIGHTS:")
    print("-" * 40)

    sktr_diff = group1['sktr_accuracy'] - group2['sktr_accuracy']
    argmax_diff = group1['argmax_accuracy'] - group2['argmax_accuracy']

    if abs(sktr_diff) > 0.01:
        if sktr_diff > 0:
            print(f"📈 Group 1 has better SKTR performance by {sktr_diff:.4f}")
        else:
            print(f"📈 Group 2 has better SKTR performance by {-sktr_diff:.4f}")

    if abs(argmax_diff) > 0.01:
        if argmax_diff > 0:
            print(f"📊 Group 1 has better argmax performance by {argmax_diff:.4f}")
        else:
            print(f"📊 Group 2 has better argmax performance by {-argmax_diff:.4f}")

    sktr_vs_argmax_g1 = group1['sktr_accuracy'] - group1['argmax_accuracy']
    sktr_vs_argmax_g2 = group2['sktr_accuracy'] - group2['argmax_accuracy']

    if abs(sktr_vs_argmax_g1) > 0.01:
        if sktr_vs_argmax_g1 > 0:
            print(f"🔬 In Group 1, SKTR outperforms argmax by {sktr_vs_argmax_g1:.4f}")
        else:
            print(f"🔬 In Group 1, argmax outperforms SKTR by {-sktr_vs_argmax_g1:.4f}")

    if abs(sktr_vs_argmax_g2) > 0.01:
        if sktr_vs_argmax_g2 > 0:
            print(f"🔬 In Group 2, SKTR outperforms argmax by {sktr_vs_argmax_g2:.4f}")
        else:
            print(f"🔬 In Group 2, argmax outperforms SKTR by {-sktr_vs_argmax_g2:.4f}")

    return group_stats

# Run the analysis
csv_path = "recovery_results_50salads_complete_15.csv"
group1_cases_str = ['20', '11', '5', '36', '14', '4', '30', '15', '3', '18']

# Get all cases from the CSV to determine group2
results_df = pd.read_csv(csv_path)
all_csv_cases = results_df['case:concept:name'].unique()

# Convert string case IDs to integers for CSV filtering (CSV stores them as integers)
group1_cases = [int(case_id) for case_id in group1_cases_str]
group2_cases = [int(case_id) for case_id in all_csv_cases if case_id not in group1_cases]

print(f"Found {len(all_csv_cases)} total cases in CSV")
print(f"Group 1: {len(group1_cases)} cases {group1_cases_str}")
print(f"Group 2: {len(group2_cases)} cases")
print(f"Note: CSV stores case IDs as integers, so we convert strings to integers for filtering")

# Run the analysis
results_stats = analyze_results_from_csv(csv_path, group1_cases, group2_cases)


Found 40 total cases in CSV
Group 1: 10 cases ['20', '11', '5', '36', '14', '4', '30', '15', '3', '18']
Group 2: 30 cases
Note: CSV stores case IDs as integers, so we convert strings to integers for filtering
ANALYSIS OF RESULTS FROM CSV FILE
Loading results from: recovery_results_50salads_complete_15.csv
Loaded 237820 rows from CSV file
Unique cases in results: 40

GROUP COMPARISON RESULTS

1. SKTR ALGORITHM PERFORMANCE:
----------------------------------------
Group 1 - Mean SKTR accuracy: 0.8306
Group 2 - Mean SKTR accuracy: 0.7915
SKTR difference: 0.0391

2. ARGMAX ALGORITHM PERFORMANCE:
----------------------------------------
Group 1 - Mean argmax accuracy: 0.8295
Group 2 - Mean argmax accuracy: 0.8247
Argmax difference: 0.0048

3. ALGORITHM COMPARISON WITHIN GROUPS:
----------------------------------------
Group 1 - SKTR vs Argmax: 0.0011
Group 2 - SKTR vs Argmax: -0.0332

4. TOP 5 INDIVIDUAL PERFORMANCES:
----------------------------------------
Group 1 SKTR (highest to lowest)

In [18]:
# KARI Results Analysis (mean ± std and micro accuracy by groups)
# Expects `df`, `group1_cases_str` to be defined (from earlier cells)

import pickle
import numpy as np
from typing import List

# Path to KARI results
kari_pkl_path = "kari_results_50salads_complete.pkl"

# Determine case ID order for KARI predictions
# Default order for 50salads (as used in utils)
default_case_order: List[str] = [
    '30', '17', '9', '8', '20', '7', '23', '5', '28', '2', '1', '0',
    '13', '36', '33', '3', '14', '10', '31', '22', '34', '38', '37', '6',
    '24', '27', '21', '15', '11', '19', '16', '12', '32', '25', '35', '39',
    '26', '29', '4', '18'
]

# Use DF case IDs as a fallback order if needed
all_cases_df = [str(x) for x in df['case:concept:name'].unique().tolist()]

# Ensure group definitions in string form
try:
    group1_cases_str
except NameError:
    group1_cases_str = ['20', '11', '5', '36', '14', '4', '30', '15', '3', '18']

group2_cases_str = [cid for cid in all_cases_df if cid not in group1_cases_str]

print("=" * 80)
print("KARI RESULTS ANALYSIS (from pickle)")
print("=" * 80)
print(f"Loading: {kari_pkl_path}")

with open(kari_pkl_path, 'rb') as f:
    kari_results = pickle.load(f)

# Choose case order: prefer default if lengths match, else DF order
if len(kari_results) == len(default_case_order):
    case_id_order = default_case_order
else:
    case_id_order = all_cases_df
    if len(kari_results) != len(case_id_order):
        raise ValueError(f"KARI results length ({len(kari_results)}) does not match number of DF cases ({len(case_id_order)})")

# Build ground-truth sequences by case (strings)
from utils import get_sequences_by_case
gt_sequences = get_sequences_by_case(df, case_id_order)

# Extract predictions per case from KARI results
pred_sequences = []
for i, result in enumerate(kari_results):
    if 'labels' not in result:
        raise ValueError(f"KARI result at index {i} missing 'labels'")
    labels_array = result['labels']
    pred_seq = [str(x) for x in labels_array]
    pred_sequences.append(pred_seq)

# Validate lengths
for i, (gt_seq, pred_seq) in enumerate(zip(gt_sequences, pred_sequences)):
    if len(gt_seq) != len(pred_seq):
        raise ValueError(f"Sequence length mismatch for case {case_id_order[i]}: GT={len(gt_seq)}, Pred={len(pred_seq)}")

# Per-case accuracies and micro (frame) counts
case_to_acc = {}
case_to_len = {}
correct_counts = { 'group1': 0, 'group2': 0 }
total_counts   = { 'group1': 0, 'group2': 0 }

for i, case_id in enumerate(case_id_order):
    gt_seq = gt_sequences[i]
    pr_seq = pred_sequences[i]
    correct = sum(1 for g, p in zip(gt_seq, pr_seq) if g == p)
    acc = correct / len(gt_seq) if gt_seq else 0.0
    case_to_acc[case_id] = acc
    case_to_len[case_id] = len(gt_seq)

    grp = 'group1' if case_id in group1_cases_str else 'group2'
    correct_counts[grp] += correct
    total_counts[grp]   += len(gt_seq)

# Aggregate stats
def summarize_group_kari(group_case_ids: List[str]):
    per_case = [case_to_acc[cid] for cid in group_case_ids if cid in case_to_acc]
    mean_acc = float(np.mean(per_case)) if per_case else float('nan')
    std_acc  = float(np.std(per_case))  if per_case else float('nan')
    return mean_acc, std_acc, per_case

g1_mean, g1_std, g1_list = summarize_group_kari(group1_cases_str)
g2_mean, g2_std, g2_list = summarize_group_kari(group2_cases_str)

# Micro accuracies
g1_micro = correct_counts['group1'] / total_counts['group1'] if total_counts['group1'] > 0 else float('nan')
g2_micro = correct_counts['group2'] / total_counts['group2'] if total_counts['group2'] > 0 else float('nan')

# Print summary
print("\n1. KARI CASE-LEVEL MEAN ± STD:")
print("-" * 40)
print(f"Group 1 KARI: {g1_mean:.4f} ± {g1_std:.4f} (n={len(g1_list)})")
print(f"Group 2 KARI: {g2_mean:.4f} ± {g2_std:.4f} (n={len(g2_list)})")

print("\n2. KARI MICRO ACCURACY (OVER FRAMES):")
print("-" * 40)
print(f"Group 1 KARI micro: {g1_micro:.4f}")
print(f"Group 2 KARI micro: {g2_micro:.4f}")

# Optional: top-5 per-group
print("\n3. TOP 5 INDIVIDUAL KARI ACCURACIES PER GROUP:")
print("-" * 40)
print("Group 1 (highest to lowest):")
for i, acc in enumerate(sorted(g1_list, reverse=True)[:5]):
    print(f"  {i+1}. {acc:.4f}")
print("Group 2 (highest to lowest):")
for i, acc in enumerate(sorted(g2_list, reverse=True)[:5]):
    print(f"  {i+1}. {acc:.4f}")


KARI RESULTS ANALYSIS (from pickle)
Loading: kari_results_50salads_complete.pkl

1. KARI CASE-LEVEL MEAN ± STD:
----------------------------------------
Group 1 KARI: 0.7677 ± 0.1204 (n=10)
Group 2 KARI: 0.8021 ± 0.0974 (n=30)

2. KARI MICRO ACCURACY (OVER FRAMES):
----------------------------------------
Group 1 KARI micro: 0.7686
Group 2 KARI micro: 0.8019

3. TOP 5 INDIVIDUAL KARI ACCURACIES PER GROUP:
----------------------------------------
Group 1 (highest to lowest):
  1. 0.9676
  2. 0.9088
  3. 0.8805
  4. 0.8387
  5. 0.7527
Group 2 (highest to lowest):
  1. 0.9713
  2. 0.9675
  3. 0.9270
  4. 0.9182
  5. 0.9111


In [16]:
# Save output list as test matrices for later use
import pickle

test_matrices = output
with open("test_matrices.pkl", "wb") as f:
    pickle.dump(test_matrices, f)

In [9]:
# standard window-basedconformance 
accuracies = compute_accuracies_by_case(results_df)

accuracies

,case:concept:name,sktr_accuracy,argmax_accuracy
0,11,0.807453,0.813665
1,14,0.884354,0.884354
2,15,0.870748,0.863946
3,18,0.766917,0.699248
4,20,0.941558,0.902597
5,3,0.968254,0.952381
6,30,0.896825,0.904762
7,36,0.676190,0.666667
8,4,0.662857,0.657143
9,5,0.685714,0.668571


In [16]:
prob_dict[('1',)]

{'1': 0.86,
 '0': 0.03,
 '2': 0.03,
 '11': 0.02,
 '15': 0.03,
 '4': 0.03,
 '13': 0.01}

In [10]:
# Show all rows for case '28' without truncation, including full list values
case_28_df = results_df[results_df['case:concept:name'] == '36']
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None, 'display.width', None):
    display(case_28_df)
total_cost = case_28_df['sktr_move_cost'].sum()
print(f"Total SKTR move cost for case 28: {total_cost:.4f}")

,case:concept:name,step,sktr_activity,argmax_activity,ground_truth,all_probs,all_activities,is_correct,cumulative_accuracy,sktr_move_cost
490,36,0,17,17,17,"[0.0, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.03, 0.0, 0.01, 0.0, 0.01, 0.0, 0.88, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.12
491,36,1,17,17,17,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.01, 0.0, 0.0, 0.0, 0.01, 0.0, 0.94, 0.0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.06
492,36,2,17,17,17,"[0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.0, 0.91, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.09
493,36,3,17,17,17,"[0.05, 0.01, 0.03, 0.02, 0.02, 0.02, 0.03, 0.02, 0.09, 0.05, 0.02, 0.07, 0.02, 0.03, 0.01, 0.01, 0.01, 0.47, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.53
494,36,4,17,17,17,"[0.02, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01, 0.02, 0.01, 0.01, 0.03, 0.0, 0.02, 0.0, 0.01, 0.0, 0.8, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.20
495,36,5,17,17,17,"[0.02, 0.01, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01, 0.04, 0.01, 0.01, 0.0, 0.01, 0.0, 0.82, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.18
496,36,6,17,17,17,"[0.02, 0.01, 0.02, 0.01, 0.01, 0.02, 0.02, 0.01, 0.07, 0.01, 0.02, 0.05, 0.01, 0.02, 0.0, 0.01, 0.0, 0.65, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.35
497,36,0,17,17,0,"[0.19, 0.01, 0.03, 0.02, 0.01, 0.02, 0.02, 0.02, 0.06, 0.02, 0.04, 0.13, 0.04, 0.02, 0.0, 0.02, 0.01, 0.29, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.875000,0.71
498,36,1,0,0,0,"[0.65, 0.03, 0.03, 0.01, 0.02, 0.01, 0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.05, 0.01, 0.01, 0.01, 0.02, 0.02, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,0.888889,1.21
499,36,2,0,0,0,"[0.69, 0.03, 0.02, 0.01, 0.02, 0.01, 0.0, 0.01, 0.02, 0.01, 0.06, 0.02, 0.05, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,0.900000,0.31


Total SKTR move cost for case 28: 56.9400


# Results Investigation

In [10]:
# Comprehensive Group Statistics Analysis
# This cell computes detailed statistics to understand differences between the two groups

def print_group_comparison(stats):
    """Print a comprehensive comparison of two groups"""
    print("=" * 80)
    print("COMPREHENSIVE GROUP COMPARISON ANALYSIS")
    print("=" * 80)

    group1 = stats['group1']
    group2 = stats['group2']

    print("\n1. ARGMAX ACCURACY COMPARISON:")
    print("-" * 40)
    print(f"Group 1 - Mean: {group1['mean_accuracy']:.4f} ± {group1['std_accuracy']:.4f}")
    print(f"Group 2 - Mean: {group2['mean_accuracy']:.4f} ± {group2['std_accuracy']:.4f}")
    print(f"Difference: {group1['mean_accuracy'] - group2['mean_accuracy']:.4f}")

    print("\n2. SEQUENCE LENGTH ANALYSIS:")
    print("-" * 40)
    print(f"Group 1 - Mean length: {group1['mean_sequence_length']:.1f} ± {group1['std_sequence_length']:.1f}")
    print(f"Group 2 - Mean length: {group2['mean_sequence_length']:.1f} ± {group2['std_sequence_length']:.1f}")
    print(f"Length difference: {group1['mean_sequence_length'] - group2['mean_sequence_length']:.1f}")

    print("\n3. PREDICTION CONFIDENCE ANALYSIS:")
    print("-" * 40)
    print(f"Group 1 - Mean max prob: {group1['mean_max_prob']:.4f} ± {group1['std_max_prob']:.4f}")
    print(f"Group 2 - Mean max prob: {group2['mean_max_prob']:.4f} ± {group2['std_max_prob']:.4f}")
    print(f"Confidence difference: {group1['mean_max_prob'] - group2['mean_max_prob']:.4f}")

    print("\n4. ENTROPY ANALYSIS:")
    print("-" * 40)
    print(f"Group 1 - Mean entropy: {group1['mean_entropy']:.4f} ± {group1['std_entropy']:.4f}")
    print(f"Group 2 - Mean entropy: {group2['mean_entropy']:.4f} ± {group2['std_entropy']:.4f}")
    print(f"Entropy difference: {group1['mean_entropy'] - group2['mean_entropy']:.4f}")
    print("(Lower entropy = more confident predictions)")

    # Class distribution analysis
    print("\n5. ACTIVITY CLASS DISTRIBUTIONS:")
    print("-" * 40)

    # Get all unique classes
    all_classes = set(group1['gt_class_distribution'].keys()) | set(group2['gt_class_distribution'].keys())

    print("Ground Truth Class Distribution:")
    print("Class | Group1 Count | Group1 % | Group2 Count | Group2 % | Difference")
    print("-" * 70)

    for cls in sorted(all_classes):
        g1_count = group1['gt_class_distribution'].get(cls, 0)
        g2_count = group2['gt_class_distribution'].get(cls, 0)

        g1_total = sum(group1['gt_class_distribution'].values())
        g2_total = sum(group2['gt_class_distribution'].values())

        g1_pct = g1_count / g1_total * 100 if g1_total > 0 else 0
        g2_pct = g2_count / g2_total * 100 if g2_total > 0 else 0

        print(f"{cls:5} | {g1_count:11} | {g1_pct:7.2f} | {g2_count:11} | {g2_pct:7.2f} | {g1_pct - g2_pct:8.2f}")

    print("\n6. TOP 5 ACCURACIES PER GROUP:")
    print("-" * 40)
    print("Group 1 (highest to lowest):")
    for i, acc in enumerate(sorted(group1['individual_accuracies'], reverse=True)[:5]):
        print(f"  {i+1}. {acc:.4f}")

    print("Group 2 (highest to lowest):")
    for i, acc in enumerate(sorted(group2['individual_accuracies'], reverse=True)[:5]):
        print(f"  {i+1}. {acc:.4f}")

# Run the comprehensive analysis
from utils import compute_comprehensive_group_statistics

# Define the two groups
group1_cases = ['20', '11', '5', '36', '14', '4', '30', '15', '3', '18']
group2_cases = [str(i) for i in range(len(softmax_lst)) if str(i) not in group1_cases]

print(f"Analyzing Group 1: {len(group1_cases)} cases")
print(f"Analyzing Group 2: {len(group2_cases)} cases")

# Compute comprehensive statistics
stats = compute_comprehensive_group_statistics(df, softmax_lst, group1_cases, group2_cases)

# Print comparison
print_group_comparison(stats)

print("\n" + "=" * 80)
print("ANALYSIS SUMMARY")
print("=" * 80)

if stats['group1']['mean_accuracy'] - stats['group2']['mean_accuracy'] > 0.01:
    print("📈 Group 1 has noticeably higher argmax accuracy")
elif stats['group2']['mean_accuracy'] - stats['group1']['mean_accuracy'] > 0.01:
    print("📈 Group 2 has noticeably higher argmax accuracy")
else:
    print("📊 Argmax accuracies are very similar between groups")

if abs(stats['group1']['mean_sequence_length'] - stats['group2']['mean_sequence_length']) > 20:
    print("📏 Groups have substantially different sequence lengths")
else:
    print("📐 Groups have similar sequence lengths")

if abs(stats['group1']['mean_max_prob'] - stats['group2']['mean_max_prob']) > 0.05:
    print("🎯 Groups have substantially different prediction confidence")
else:
    print("🎲 Groups have similar prediction confidence")

if abs(stats['group1']['mean_entropy'] - stats['group2']['mean_entropy']) > 0.1:
    print("🔀 Groups have substantially different prediction uncertainty")
    if stats['group1']['mean_entropy'] < stats['group2']['mean_entropy']:
        print("   (Group 1 has more confident predictions)")
    else:
        print("   (Group 2 has more confident predictions)")
else:
    print("⚖️ Groups have similar prediction uncertainty")


Analyzing Group 1: 10 cases
Analyzing Group 2: 30 cases
COMPREHENSIVE GROUP COMPARISON ANALYSIS

1. ARGMAX ACCURACY COMPARISON:
----------------------------------------
Group 1 - Mean: 0.8295 ± 0.1067
Group 2 - Mean: 0.8247 ± 0.0962
Difference: 0.0048

2. SEQUENCE LENGTH ANALYSIS:
----------------------------------------
Group 1 - Mean length: 5898.9 ± 389.2
Group 2 - Mean length: 5961.0 ± 365.1
Length difference: -62.1

3. PREDICTION CONFIDENCE ANALYSIS:
----------------------------------------
Group 1 - Mean max prob: 0.9331 ± 0.1362
Group 2 - Mean max prob: 0.9361 ± 0.1357
Confidence difference: -0.0031

4. ENTROPY ANALYSIS:
----------------------------------------
Group 1 - Mean entropy: 0.2628 ± 0.4249
Group 2 - Mean entropy: 0.2482 ± 0.4090
Entropy difference: 0.0146
(Lower entropy = more confident predictions)

5. ACTIVITY CLASS DISTRIBUTIONS:
----------------------------------------
Ground Truth Class Distribution:
Class | Group1 Count | Group1 % | Group2 Count | Group2 % | Dif